# F1) Automated MD&A Draft from Financials (RAG + Summarization)

This notebook loads tabular financial statement extracts, computes YoY/QoQ deltas and KPIs, chunks filings/statements into retrievable text snippets, builds a vector index (Chroma), and uses Gemini to generate a sectioned MD&A draft (Trends, Revenue Drivers, Risks) with citations.

Prereqs:
- Place your CSV extracts under a data directory (e.g., `../backend/data/financials/`).
- Ensure environment has `GEMINI_API_KEY`.
- Dependencies: pandas, numpy, chromadb, sentence-transformers, google-generativeai.


In [ ]:
# 1) Install and Imports

# If running in a fresh environment, uncomment and run the following to install missing packages.
# %pip install pandas numpy chromadb sentence-transformers google-generativeai python-dotenv

import os
import json
import pandas as pd
import numpy as np

from dotenv import load_dotenv

# Display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 180)

# Local pipeline utilities
import importlib
from pathlib import Path

# Ensure we can import the pipeline module
root_dir = Path(__file__).resolve().parents[1]
nb_dir = Path(__file__).resolve().parent
if str(nb_dir) not in os.sys.path:
    os.sys.path.insert(0, str(nb_dir))

pipeline = importlib.import_module('mda_pipeline')
print('Loaded pipeline from:', pipeline.__file__)


In [ ]:
# 2) Configuration and File Paths

load_dotenv()

DATA_DIR = os.getenv('MDA_DATA_DIR', str((root_dir / 'backend' / 'data').resolve()))
COMPANY = os.getenv('MDA_COMPANY', None)  # e.g., 'AAPL'
FILE_PATTERN = os.getenv('MDA_FILE_PATTERN', '*.csv')

# Model / vector settings
EMBED_MODEL = os.getenv('MDA_EMBED_MODEL', 'all-MiniLM-L6-v2')
PERSIST_DIR = os.getenv('MDA_PERSIST_DIR', str((root_dir / 'chromadb').resolve()))

# Output path
OUT_DIR = os.getenv('MDA_OUT_DIR', str((root_dir / 'backend' / 'data' / 'outputs').resolve()))
os.makedirs(OUT_DIR, exist_ok=True)
OUT_MD_PATH = os.path.join(OUT_DIR, f'mda_{COMPANY or "company"}.md')

print('DATA_DIR    :', DATA_DIR)
print('COMPANY     :', COMPANY)
print('FILE_PATTERN:', FILE_PATTERN)
print('PERSIST_DIR :', PERSIST_DIR)
print('OUT_MD_PATH :', OUT_MD_PATH)


In [ ]:
# 3) Load Financial Statements (CSV)

raw_df = pipeline.load_financials(
    data_dir=DATA_DIR,
    file_pattern=FILE_PATTERN,
    company_col='company',
    period_col='period',
    concept_col='concept',
    value_col='value',
)
print('Raw rows:', len(raw_df))
raw_df.head(10)

In [ ]:
# 4-5) Normalize Periods and Compute YoY/QoQ Deltas & KPIs

kpi_df = pipeline.compute_kpis_and_deltas(raw_df, company=COMPANY)
print('KPI rows:', len(kpi_df))
kpi_df.tail(8)

In [ ]:
# 6) Minimal Unit Tests for KPI Math

# Create a tiny synthetic time series to validate QoQ ~10% growth
import math

_ts = pd.Series([100, 110, 121, 133.1, 146.41])  # ~10% compounded
_qoq = _ts.pct_change().round(3)
assert abs(_qoq.iloc[1] - 0.10) < 0.02, 'QoQ should be ~10% at step 1'
print('Synthetic QoQ test passed:', _qoq.tolist())


In [ ]:
# 7-9) Chunk KPI rows into text and build vector index (Chroma)

chunks = pipeline.build_text_chunks(kpi_df)
print('Chunks:', len(chunks))
client, collection = pipeline.build_index(chunks, persist_path=PERSIST_DIR)
print('Collection ready')

In [ ]:
# 10-12) Configure retriever and draft sections with citations using Gemini

# Sanity retrieval
sample_query = f"Key trends and YoY/QoQ performance for {COMPANY or 'the company'}"
res = pipeline.retrieve(collection, sample_query, top_k=5)
res_keys = {k: len(v[0]) if isinstance(v, list) else type(v) for k, v in res.items()}
print('Retrieve keys:', res_keys)

# Draft full MD&A
md_text = pipeline.draft_mdna(collection, company=COMPANY)
print(md_text[:800])


In [ ]:
# 13-14) Assemble and persist outputs

# Save markdown draft
with open(OUT_MD_PATH, 'w', encoding='utf-8') as f:
    f.write(md_text)
print('Saved:', OUT_MD_PATH)

# Optional: save a JSON sidecar with basic info
sidecar = {
    'company': COMPANY,
    'out_md': OUT_MD_PATH,
    'data_dir': DATA_DIR,
}
sidecar_path = OUT_MD_PATH.replace('.md', '.json')
with open(sidecar_path, 'w', encoding='utf-8') as f:
    json.dump(sidecar, f, indent=2)
print('Saved:', sidecar_path)
